In [14]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver

## chrome실행
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(5)
driver.get("https://sports.koreanpc.kr/front/club/listClub.do;jsessionid=0E47FE540D3B9FC0C51A3B4D4B356314")

In [15]:
##등록년도 조회
def lookup(year):
    input_text = driver.find_element_by_id('searchYear')
    input_text.clear()
    input_text.send_keys(year)
    driver.find_element_by_class_name("btn_v6").click()
lookup(2020)

In [26]:
##마지막 페이지 찾기
def find_last_page():
    page_bar = driver.find_elements_by_class_name("paging_area")[0]
    pages = page_bar.find_elements_by_tag_name('a')[-1].click()
    page_bar = driver.find_elements_by_class_name("paging_area")[0]
    last_page = page_bar.find_elements_by_tag_name('a')[-3].text
    return int(last_page)

In [28]:
## 마지막 페이지 저장 후 처음으로 이동(초기 셋팅)
last_page = find_last_page()
page_bar = driver.find_elements_by_class_name("paging_area")[0]
page_bar.find_elements_by_tag_name('a')[0].click()

In [32]:
#반복하여 페이지 이동
BARCOUNT = 2
while(True): 
    current_page = int(driver.find_element_by_class_name("current").text)
    if current_page >= last_page:
        break
    page_index = current_page%10+BARCOUNT
    page_bar = driver.find_elements_by_class_name("paging_area")[0]
    page_bar.find_elements_by_tag_name('a')[page_index].click()
    if current_page % 10 == 0:    ## 모든 함수 수행 후 실행해야 함
        page_bar = driver.find_elements_by_class_name("paging_area")[0]
        page_bar.find_elements_by_tag_name('a')[-2].click()

KeyboardInterrupt: 

In [ ]:
html = driver.page_source
result = bs(html, 'lxml')#==html.parser

In [ ]:
columns = ['번호','지역','클럽명','활동시간','종목','기타종목','장애유형','승인일']
result1 = result.find_all('td',{'data-cell-header':columns})
result2 = list(map(lambda x : x.text.strip(),result1))

In [ ]:
dic = {}
for col in columns:
    dic[col] = list()
count = 0
for i in range(len(result2)):
    if i > 0 and i % 8 == 0:
        count -= 8
    dic[columns[count]].append(result2[i]) 
    count+=1

dic

In [ ]:
dic['센터명'] = ['생활체육정보센터']*len(dic['번호'])
fir_df = pd.DataFrame(dic,columns=dic.keys(), index=dic['번호'])
fir_df= fir_df.drop('번호',axis=1)
fin_df = fir_df.sort_index(ascending=True)

In [ ]:
fin_df